In [1]:
HOST = "localhost:3000"

PROJECT_NAME = "CustomerChurn"
EXPERIMENT_NAME = "DecisionTreeClassifier"

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('../../data/data.csv')


In [4]:
# do all data massaging
# Convert binary variable into numeric so plotting is easier. We need to later take mean
data['Churn'] = data['Churn'].map({'Yes': 1, 'No': 0})

data.replace(" ", np.nan, inplace=True)

data['TotalCharges'] = pd.to_numeric(data['TotalCharges'])

mean = data['TotalCharges'].mean()
data.fillna(mean, inplace=True)

import category_encoders as ce
import joblib

names = ['gender', 'Partner', 'Dependents', 'PhoneService', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling']
# for column in names:
#     labelencoder(column)
data_enc = data
data_enc = data_enc.drop(['Churn', 'customerID'], axis=1)
enc = ce.ordinal.OrdinalEncoder(cols=names)
enc.fit(data_enc)
labelled_set = enc.transform(data_enc)

names = ['MultipleLines', 'InternetService', 'Contract', 'PaymentMethod', 'OnlineSecurity', 'OnlineBackup',
         'DeviceProtection', 'TechSupport']

ohe = ce.OneHotEncoder(cols=names)
data_ohe = data
data_ohe = data_ohe.drop(['Churn', 'customerID'], axis=1)
ohe.fit(data_ohe)
final_set = ohe.transform(labelled_set)

final_set.head(5)

/Users/faisalmasood/.pyenv/versions/3.8.11/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines_1,MultipleLines_2,MultipleLines_3,InternetService_1,...,Contract_1,Contract_2,Contract_3,PaperlessBilling,PaymentMethod_1,PaymentMethod_2,PaymentMethod_3,PaymentMethod_4,MonthlyCharges,TotalCharges
0,1,0,1,1,1,1,1,0,0,1,...,1,0,0,1,1,0,0,0,45.65,45.65
1,1,0,2,2,4,1,0,1,0,0,...,1,0,0,2,1,0,0,0,101.15,385.90
2,2,1,1,1,17,1,1,0,0,0,...,0,1,0,2,0,1,0,0,20.65,330.60
3,1,0,1,1,22,2,0,0,1,1,...,0,1,0,1,0,0,1,0,43.75,903.60
4,2,0,2,2,70,1,1,0,0,1,...,0,1,0,2,0,0,0,1,74.10,5222.30


In [5]:
from sklearn.model_selection import train_test_split
labels = data['Churn']
X_train, X_test, y_train, y_test = train_test_split(final_set, labels, test_size=0.2)
print ('Training Data Shape',X_train.shape, y_train.shape)
print ('Testing Data Shape',X_test.shape, y_test.shape)

Y = data['Churn']
X = final_set



Training Data Shape (5634, 36) (5634,)
Testing Data Shape (1409, 36) (1409,)


In [13]:
from verta import Client
from verta.utils import ModelAPI

client = Client(HOST)
proj = client.set_project(PROJECT_NAME)
expt = client.set_experiment(EXPERIMENT_NAME)

connection successfully established
got existing Project: CustomerChurn
got existing Experiment: DecisionTreeClassifier


In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
# grid = {
#     'max_depth': [5],
#     'criterion': ['gini'],
#     'min_samples_leaf': [3] ,
#     'min_samples_split':[5]
# }

grid = {
    'max_depth': [3,5,7,10],
    'criterion': ['gini'],
    'min_samples_leaf': [3, 5] ,
    'min_samples_split':[5,10]
}

grid_search = GridSearchCV(model, grid, return_train_score=False)
grid_search.fit(X_train, y_train)


GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini'], 'max_depth': [3, 5, 7, 10],
                         'min_samples_leaf': [3, 5],
                         'min_samples_split': [5, 10]})

In [15]:
from verta.dataset import Path, S3

results = pd.DataFrame(grid_search.cv_results_)

dataset = client.get_or_create_dataset(name="customer_churn_dataset",  desc="Customer Churn Dataset")
dataset_version = dataset.create_version(S3("s3://{}/{}".format("file", "AB_NYC_2019.csv")))


for _, run_result in results.iterrows():
    run = client.set_experiment_run()

    # log training data
    run.log_dataset("train_data", dataset_version)

    # log hyperparameters
    run.log_hyperparameters(run_result['params'])

    # log accuracy for each validation fold
    for obs_key in ["split{}_test_score".format(i) for i in range(5)]:
        run.log_observation("fold_acc", run_result[obs_key])

    # log summary stats of validation
    run.log_metric("val_acc_mean", run_result['mean_test_score'])
    run.log_metric("val_acc_std", run_result['std_test_score'])

    run.log_dataset_version("train", dataset_version)
    run.log_code()

created new Dataset: customer_churn_dataset in workspace: personal
created new Dataset Version: 1 for customer_churn_dataset
created new ExperimentRun: Run 1489916303982083816829
Git repository successfully located at /Users/faisalmasood/Dropbox/noindex/tensorflow/github/mlops-workshop/
created new ExperimentRun: Run 1489916303982091712298
Git repository successfully located at /Users/faisalmasood/Dropbox/noindex/tensorflow/github/mlops-workshop/
created new ExperimentRun: Run 1489916303982097643578
Git repository successfully located at /Users/faisalmasood/Dropbox/noindex/tensorflow/github/mlops-workshop/
created new ExperimentRun: Run 148991630398210348535
Git repository successfully located at /Users/faisalmasood/Dropbox/noindex/tensorflow/github/mlops-workshop/
created new ExperimentRun: Run 148991630398210992629
Git repository successfully located at /Users/faisalmasood/Dropbox/noindex/tensorflow/github/mlops-workshop/
created new ExperimentRun: Run 1489916303982116927612
Git repo

In [16]:
best_run = expt.expt_runs.sort("metrics.val_acc_mean", descending=True)[0]
print("Validation Accuracy: {:.4f}".format(best_run.get_metric("val_acc_mean")))

best_hyperparams = best_run.get_hyperparameters()
print("Hyperparameters: {}".format(best_hyperparams))


model = DecisionTreeClassifier(**best_hyperparams)
model.fit(X_train, y_train)


train_acc = model.score(X_test, y_test)
best_run.log_metric("train_acc", train_acc)
print("Training accuracy: {:.4f}".format(train_acc))

Validation Accuracy: 0.7994
Hyperparameters: {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 5, 'min_samples_split': 10}
Training accuracy: 0.7779


In [17]:
from verta.environment import Python

best_run.log_model(model, overwrite=True)

# registered_model = client.get_or_create_registered_model(name="CustomerChurn")
# registered_model.create_standard_model(model, name="v1", environment= Python(Python.read_pip_environment()))

upload complete (model_api.json)
upload complete (custom_modules)
upload complete (model.pkl)
